# Introduction

This notebook will contain some notes of the following papers

- [An Explanatory Note on the Basel II IRB Risk Weight Functions](http://www.bis.org/bcbs/irbriskweight.pdf)  
- [The standardised approach for measuring counterparty credit risk exposures](http://www.bis.org/publ/bcbs279.pdf)
- [Comparing Alternate Methods for Calculating CVA Capital Charges under Basel III](http://www.garp.org/media/1110535/whitepaper_cvacapitalcalculation_121812.pdf)

>The Expected Loss of a portfolio is assumed to equal the proportion of obligors that might default within a 
given time frame (1 year in the Basel context), multiplied by the outstanding exposure at default, and once more multiplied by the loss given default rate (i.e. the percentage of exposure that will not be recovered by sale of collateral etc.

>The three factors mentioned above correspond to the risk parameters upon which the Basel II IRB approach is built:  
- probability of default (PD) per rating grade, which gives the average percentage of obligors that default in this rating grade in the course of one year
- exposure at default (EAD), which gives an estimate of the amount outstanding (drawn amounts plus likely future drawdowns of yet undrawn lines) in case the borrower defaults
- loss given default (LGD), which gives the percentage of exposure the bank might lose in case the borrower defaults. These losses are usually shown as a percentage of EAD, and depend, amongst others, on the type and amount of collateral as well as the type of borrower and the expected proceeds from the work-out of the assets.  

\begin{equation*}
\text{EL} = \text{PD} \times \text{EAD} \times \text{LGD}
\end{equation*}

# Regulatory capital for counterparty credit risk ($K_{CCR}$)

## Formula

The $RWA (=K_{CCR})$ is given by: 

\begin{equation*}
RWA = 12,5 \times \omega \times EAD
\end{equation*}

## Weight $\omega$

### Standardized Approach

Different tables and mapping are provided by Basel. All the following information of the SA will come from [BCBS 128](http://www.bis.org/publ/bcbs128.pdf).

- Claims on sovereigns see $\S 53$
- Claims on banks see $\S 63$
- Claims on corporates see $\S 66$
- ... and some over specific claims

More specifically, see [BCBS The Standardised Approach to Credit Risk](http://www.bis.org/publ/bcbsca04.pdf)

### IRB Approach

The Capital weight regulatory formula is given by:

\begin{equation*}
\omega = \text{LGD} \times \left( 
            \Phi \left( 
                \frac{\Phi^{-1} \left( \text{PD} \right)}{\sqrt{1-\text{R}}}
                + \sqrt{\frac{\text{R}}{1-\text{R}}} \times \Phi^{-1}(0.999)
            \right) 
            - \text{PD} \right)
        \times \frac{1 + ( \text{M} - 2.5) \times \text{b(PD)}}{1 - 1.5 \times \text{b(PD)}}
\end{equation*}

where:

- $\Phi$ is the CDF of the standard normal distribution
- $\text{R}$ is the assets - counterparties correlation
    - $\text{R(PD)} = 0.12 \frac{1-e^{-50 \times PD}}{1-e^{-50}} + 0.24 \frac{1 - (1 - e^{-50 \times PD})}{1-e^{-50}}$
- $\text{M}$ is the eﬀective maturity of the netting set
    - $\text{M} = \min(5.0, \max(1.0, \frac{\sum_{i=1}^N m_i \, N_i}{\sum_{i=1}^N N_i})$
- $\text{b}$ is the smoothed (regression) maturity adjustment (smoothed over PDs)
    - $\text{b}(x) = \Big( 0.11852 - 0.05478 \times \ln x \Big)^2$

## Exposure at default $EAD$

### Standardized Approach

The Exposure at default ($EAD$) is given by:

\begin{equation*}
\text{EAD} = 1.4 \times (\text{RC} + \text{PFE})
\end{equation*}

with:

- $\text{RC}$ the replacement cost
- $\text{PFE}$ the amount for potential future exposure

In the code, we will put a $\text{PFE}$ value to $0$ (too much complicated to handle the different derivatives). Moreover, instead of using the BCBS valuation for the $\text{RC}$, we will use $\text{RC} = (\text{V} - \text{Collat})^+$ (see $\S$ 139-145 of 2nd reference above)
Thus, in our framework,

\begin{equation*}
\text{EAD} = 1.4 \times (\text{V}-\text{Collat})^+
\end{equation*}

# Regulatory capital for CVA in Basel III ($K_{CVA}$)

>(From Kenyon) The standardized CVA risk capital charge in (BCBS-189 2011), $\S$ 104, gives the formula to generate CVA capital (in the absence of hedging):

\begin{equation*}
\text{K}_{CVA} = 
    2.33 \sqrt{h} \left[
        \left( 0.5 \sum_{i=1}^n w_i M_i \tilde{EAD_i}^{tot} \right)^2
        + 0.75 \sum_{i=1}^n \left( w_i M_i \tilde{EAD_i}^{tot} \right)^2
    \right]^{0.5}
\end{equation*}

As in Kenyon, we will use the following formula towards the $i$-th counterparty:

\begin{equation*}
\text{K}_{CVA}^i = 
    \frac{2.33}{2} \sqrt{h} w_i M_i \frac{1-e^{-0.05M_i}}{0.05 M_i} EAD_i^{tot}
\end{equation*}


where:

- $h$ is the one year risk horizon i.e., $h=1$
- $w_i$ is the risk weight of th $i$th counterparty, based on external rating or equivalent
- $\tilde{EAD_i}$ is the exposure at default of counterparty $i$, discounted using $\frac{1 - \exp(0.05M_i)}{0.05M_i}$
- $M_i$ is the effective maturity of the of the netting set with counterparty $i$ as above
- $w_i$ is the weight based on the external rating (see $\S 104$) of [BCBS 189](http://www.bis.org/publ/bcbs189.pdf):

\begin{array}{|c|c|}
\hline
\text{Rating} & \text{Weight} \\
\hline
\text{AAA} & 0.7\% \\
\text{AA} & 0.7\% \\
\text{A} & 0.8\% \\
\text{BBB} & 1.0\% \\
\text{BB} & 2.0\% \\
\text{B} & 3.0\% \\
\text{CCC} & 10.0\% \\
\hline
\end{array}

Moreover, we have the following study from Moody's:

\begin{array}{|c|c|}
\hline
\text{Rating} & \text{Default Prob} \\
\hline
\text{AAA} & 0.00\% \\
\text{AA} & 0.02\% \\
\text{A} & 0.06\% \\
\text{BBB} & 0.17\% \\
\text{BB} & 1.06\% \\
\text{B} & 3.71\% \\
\text{CCC} & 12.81\% \\
\hline
\end{array}